In [1]:
!pip install uproot awkward 
from uproot_io import Events, View
import numpy as np
import matplotlib.pyplot as plt
import features as u

cheated_0 = Events('CheatedRecoFile_0.root')

# we have 499617 events in cheated_0

In [ ]:
def var_adc_max(events, start_idx, end_idx, hits_cutoff, show_plot=False):

    if hits_cutoff < 2:
        hits_cutoff = 2
        print('cannot accept events with less than 2 hits')
    # define an array for the max values that we will plot with eventually
    max_array_shower = []
    max_array_track = []

    # add a counter for the fails and successes, just so the debugging data isnt as invasive
    f = 0
    n = 0
    
    for event_idx in range(start_idx, end_idx + 1):

        # find w positions so we can relate the adc values
        w_hits = events.reco_hits_w[event_idx]

        if len(w_hits) > hits_cutoff:
            
            adc_values = events.reco_adcs_w[event_idx]
        
            # filtering track or shower from pdg code
            pdg = events.mc_pdg[event_idx]
            shower = False # initialising the track or shower variable

            if pdg in [-11, 11, 22]:
                shower = True

            # finding the w value associated with the maximum adc value
            w_idx = np.argmax(adc_values)

            # we are omitting an edge case here that there is a chance there are two identical max values, this will rarely happen though

            # want to scale the w_adc_max value
            w_adc_max = w_hits[w_idx]
            # in the samw way we scale the w hits, translate to origin then squash to [0,1]:

            w_adc_max = (w_adc_max - min(w_hits))/(max(w_hits) - min(w_hits))
            n += 1

            if shower:
                max_array_shower.append(w_adc_max)
            else: max_array_track.append(w_adc_max)

        else: f += 1

    print(f'There were {f} failed indices')
    
    edges = np.linspace(0,1,21)
    plt.figure(figsize=(10,6))
    plt.hist(max_array_track, density=True, color='c', edgecolor='k', bins=edges, linewidth=0.5, alpha=0.7, label='True Track')
    plt.hist(max_array_shower, density=True, color='m', edgecolor='k', bins=edges, linewidth=0.5, alpha=0.7, label='True Shower')
    plt.title(f'ADC Peak histogram for {n} events, cutoff = {hits_cutoff}')
    plt.xlabel('W position in event')
    plt.ylabel('ADC Max density')
    plt.legend()
    plt.grid(False)
    plt.show()